In [1]:
from flask import Flask, request, render_template, url_for, redirect, jsonify
import tensorflow as tf
import pandas as pd
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn import preprocessing

In [2]:
def neural_network(filename):
    x = pd.read_csv(filename)
    y = x.pop('O3')
    
    x = preprocessing.scale(x)
    model = tf.keras.models.load_model('ozone-final.model')
    pred = model.predict(x)

    mse_train = mean_squared_error(y, pred)
    rmse_train = sqrt(mean_squared_error(y, pred))
    mae_train = mean_absolute_error(y, pred)
    r2 = "{:0.3f}".format(r2_score(y, pred))

    return mse_train, rmse_train, mae_train, r2

In [3]:
ui = Flask(__name__)
x = []
@ui.route('/', methods=['POST', 'GET'])
def index():
    if request.method == 'POST':
        filename = request.form['myfile']
        mse, rmse, mae, r2 = neural_network(filename) 
        return render_template('home.html', mse_dis = mse, rmse_dis = rmse, mae_dis = mae, r2_dis = r2)
    else:
        return render_template('home.html')

if __name__ == "__main__":
    ui.run()

* Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
I0429 23:52:44.683648 12264 _internal.py:122]  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
I0429 23:53:13.875893  1248 _internal.py:122] 127.0.0.1 - - [29/Apr/2020 23:53:13] "GET / HTTP/1.1" 200 -
I0429 23:53:27.938273 13824 _internal.py:122] 127.0.0.1 - - [29/Apr/2020 23:53:27] "POST / HTTP/1.1" 200 -
